In [1]:
!nvidia-smi

Sun Apr 25 01:09:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#5.6.1. 计算设备¶
#我们可以指定用于存储和计算的设备，如CPU和GPU。默认情况下，张量是在内存中创建的，然后使用CPU计算它。
#在PyTorch中，CPU和GPU可以用torch.device('cpu')和torch.cuda.device('cuda')表示。
#应该注意的是，cpu设备意味着所有物理CPU和内存。这意味着PyTorch的计算将尝试使用所有CPU核心。
#然而，gpu设备只代表一个卡和相应的显存。如果有多个GPU，我们使用torch.cuda.device(f'cuda:{i}')来表示第 i 块GPU（ i 从0开始）。另外，gpu:0和gpu是等价的。
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x7f7617793950>)

In [3]:
#我们可以查询可用gpu的数量。
torch.cuda.device_count()

1

In [4]:
#现在我们定义了两个方便的函数，这两个函数允许我们在请求的GPU不存在的情况下运行代码。
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()。"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
#5.6.2. 张量与gpu¶
#默认情况下，张量是在CPU上创建的。我们可以查询张量所在的设备。
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [6]:
#5.6.2.1. 存储在GPU上
#有几种方法可以在GPU上存储张量。例如，我们可以在创建张量时指定存储设备。接下来，
#我们在第一个gpu上创建张量变量X。在GPU上创建的张量只消耗这个GPU的显存。
#我们可以使用nvidia-smi命令查看显存使用情况。一般来说，我们需要确保不创建超过GPU显存限制的数据。
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [7]:
#假设你至少有两个GPU，下面的代码将在第二个GPU上创建一个随机张量。
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.1091, 0.8960, 0.0766],
        [0.7553, 0.9326, 0.1911]])

In [9]:
#5.6.2.2. 复制
Z = X.cuda()
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [10]:
#现在数据在同一个GPU上（Z和Y都在），我们可以将它们相加。
Y + Z

RuntimeError: ignored

In [ ]:
#假设变量Z已经存在于第二个GPU上。如果我们还是调用Z.cuda(1)怎么办？它将返回Z，而不会复制并分配新内存。
Z.cuda(1) is Z

In [ ]:
#5.6.3. 神经网络与GPU¶
#类似地，神经网络模型可以指定设备。下面的代码将模型参数放在GPU上。
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [ ]:
#当输入为GPU上的张量时，模型将在同一GPU上计算结果。
net(X)

In [ ]:
#让我们确认模型参数存储在同一个GPU上。
net[0].weight.data.device